In [1]:
import pandas as pd

In [2]:
lvst_list = ['pigs', 'buffalo', 'cattle_dairy', 'cattle_nondairy', 'chickens', 'goats', 'horses', 'mules', 'sheep']
lvst_list

['pigs',
 'buffalo',
 'cattle_dairy',
 'cattle_nondairy',
 'chickens',
 'goats',
 'horses',
 'mules',
 'sheep']

In [3]:
def get_iran_data_only(path_name):
    df = pd.read_csv(path_name)
    df_iran_only = df[df.iso_code3 == 'IRN'].drop(columns=['Nation']).reset_index(drop=True)
    return df_iran_only


def create_df_dict():

    dataframes = {}

    for lvst_type in lvst_list:
        histdata_path = f'AFOLU/pop_lvst_initial_{lvst_type}/input_to_sisepuede/historical/pop_lvst_initial_{lvst_type}.csv'
        projdata_path =  f'AFOLU/pop_lvst_initial_{lvst_type}/input_to_sisepuede/projected/pop_lvst_initial_{lvst_type}.csv'
        histdata_iran_df = get_iran_data_only(histdata_path)
        projdata_iran_df = get_iran_data_only(projdata_path)
        projdata_iran_df = projdata_iran_df[projdata_iran_df['Year'] < 2023]

        concat_df = pd.concat([histdata_iran_df, projdata_iran_df])
        concat_df = concat_df.rename(columns={f'pop_lvst_initial_{lvst_type}':f'pop_{lvst_type}'})
        dataframes[lvst_type] = concat_df
    
    return dataframes

lvst_dataframes = create_df_dict()

In [4]:
lvst_dataframes['cattle'] = pd.merge(lvst_dataframes['cattle_dairy'], lvst_dataframes['cattle_nondairy'])
lvst_dataframes['cattle']

,iso_code3,Year,pop_cattle_dairy,pop_cattle_nondairy
0,IRN,2011,7612000.0,7612000.0
1,IRN,2012,7800000.0,7800000.0
2,IRN,2013,7936000.0,7936000.0
3,IRN,2014,4231993.0,4231993.0
4,IRN,2015,4572681.0,4572681.0
5,IRN,2016,5504972.0,5504972.0
6,IRN,2017,4879363.0,4879363.0
7,IRN,2018,5492708.0,5492708.0
8,IRN,2019,5505458.0,5505458.0
9,IRN,2020,5505458.0,5505458.0


In [5]:
lvst_dataframes['cattle'] = lvst_dataframes['cattle'].drop(columns='pop_cattle_nondairy')
lvst_dataframes['cattle'] = lvst_dataframes['cattle'].rename(columns={'pop_cattle_dairy':'pop_cattle'})
lvst_dataframes['cattle']

,iso_code3,Year,pop_cattle
0,IRN,2011,7612000.0
1,IRN,2012,7800000.0
2,IRN,2013,7936000.0
3,IRN,2014,4231993.0
4,IRN,2015,4572681.0
5,IRN,2016,5504972.0
6,IRN,2017,4879363.0
7,IRN,2018,5492708.0
8,IRN,2019,5505458.0
9,IRN,2020,5505458.0


In [6]:
for lvst_type in lvst_dataframes.keys():
    print(lvst_dataframes[lvst_type])

  iso_code3  Year  pop_pigs
0       IRN  2011       0.0
1       IRN  2012       0.0
2       IRN  2013       0.0
3       IRN  2014       0.0
4       IRN  2015       0.0
5       IRN  2016       0.0
6       IRN  2017       0.0
7       IRN  2018       0.0
8       IRN  2019       0.0
0       IRN  2020       0.0
1       IRN  2021       0.0
2       IRN  2022       0.0
  iso_code3  Year  pop_buffalo
0       IRN  2011     197000.0
1       IRN  2012     199000.0
2       IRN  2013     202000.0
3       IRN  2014     106517.0
4       IRN  2015     101056.0
5       IRN  2016     125143.0
6       IRN  2017     126765.0
7       IRN  2018     139073.0
8       IRN  2019     147632.0
0       IRN  2020     147632.0
1       IRN  2021     147632.0
2       IRN  2022     147632.0
  iso_code3  Year  pop_cattle_dairy
0       IRN  2011         7612000.0
1       IRN  2012         7800000.0
2       IRN  2013         7936000.0
3       IRN  2014         4231993.0
4       IRN  2015         4572681.0
5       IRN  2016

In [7]:
validation_df = pd.read_csv('data_for_validation/FAOSTAT_data_en_10-3-2024.csv')
validation_df.head()

,Domain,Area,Element,Item,Year,Unit,Value
0,Crops and livestock products,Iran (Islamic Republic of),Stocks,Buffalo,2011,An,197000
1,Crops and livestock products,Iran (Islamic Republic of),Stocks,Buffalo,2012,An,199000
2,Crops and livestock products,Iran (Islamic Republic of),Stocks,Buffalo,2013,An,202000
3,Crops and livestock products,Iran (Islamic Republic of),Stocks,Buffalo,2014,An,106517
4,Crops and livestock products,Iran (Islamic Republic of),Stocks,Buffalo,2015,An,101056


In [8]:
validation_df = validation_df.drop(columns=['Domain', 'Unit', 'Area', 'Element'])
validation_df = validation_df.rename(columns={'Value':'sonya_fao_value'})
validation_df.head()

,Item,Year,sonya_fao_value
0,Buffalo,2011,197000
1,Buffalo,2012,199000
2,Buffalo,2013,202000
3,Buffalo,2014,106517
4,Buffalo,2015,101056


In [9]:
validation_df['Item'] = validation_df['Item'].apply(lambda x: 'pigs' if x == 'Swine / pigs' else x)
validation_df['Item'] = validation_df['Item'].apply(lambda x: 'mules' if x == 'Mules and hinnies' else x)
validation_df['Item'] = validation_df['Item'].str.lower()

In [10]:
lvst_list = list(validation_df['Item'].unique())
lvst_list

['buffalo', 'cattle', 'chickens', 'horses', 'mules', 'sheep', 'pigs']

In [11]:
for lvst_type in lvst_list:
    # print(validation_df[validation_df.Item ==  lvst_type]['Value'])
    lvst_dataframes[lvst_type] = pd.merge(lvst_dataframes[lvst_type], validation_df[validation_df['Item'] == lvst_type])
    lvst_dataframes[lvst_type]['value_difference'] = lvst_dataframes[lvst_type]['pop_' + lvst_type] - lvst_dataframes[lvst_type]['sonya_fao_value']
    lvst_dataframes[lvst_type] = lvst_dataframes[lvst_type].drop(columns='Item')
    print(lvst_dataframes[lvst_type])
    

   iso_code3  Year  pop_buffalo  sonya_fao_value  value_difference
0        IRN  2011     197000.0           197000               0.0
1        IRN  2012     199000.0           199000               0.0
2        IRN  2013     202000.0           202000               0.0
3        IRN  2014     106517.0           106517               0.0
4        IRN  2015     101056.0           101056               0.0
5        IRN  2016     125143.0           125143               0.0
6        IRN  2017     126765.0           126765               0.0
7        IRN  2018     139073.0           136062            3011.0
8        IRN  2019     147632.0           144962            2670.0
9        IRN  2020     147632.0           171156          -23524.0
10       IRN  2021     147632.0           157764          -10132.0
11       IRN  2022     147632.0           132644           14988.0
   iso_code3  Year  pop_cattle  sonya_fao_value  value_difference
0        IRN  2011   7612000.0          7612000               0